# ETL - Mapeamento de Criminalidade no Ceará

#### Este notebook tem como objetivo carregar as bases de dados fragmentadas, obtidas da Secretaria de Segurança Pública e Defesa Social (SSPDS) do estado do Ceará, entre os anos de 2013 à 2021, transformar os dados, e gerar uma versão para estudo e exploração.
#### As fontes dizem respeito aos registros de CRIMES VIOLENTOS LETAIS E INTENCIONAIS (CVLI), e embora alguns periodos tenham sido dificeis de obter/extrair, todos foram obtidos do órgão SSPDS-CE para este estudo.

In [1]:
# Importar bibliotecas
import pandas as pd

In [2]:
# Carregar as bases de 2013 à 2021 (2021 até 06)
df_base2013a2017 = pd.read_excel(r'../data/external/sspceara/lista-de-vitimas-de-cvli-ceara-2013-a-2017.xlsx', skiprows=[0,1,2,3,4])
df_base2017a2018 = pd.read_excel(r'../data/external/sspceara/cvli-e-intervencao-policial-nos-anos-de-2017-e-2018.xlsx')
df_base2019a2020 = pd.read_excel(r'../data/external/sspceara/lista-nominal_2019-a-fev2020.xlsx', skiprows=[0,1,2,3,4])
df_base2021parcial = pd.read_excel(r'../data/external/sspceara/CVLI_2021_01a06.xls')

### Padronização das colunas
As bases serão formatadas para trabalhar com esse padrão de nominação de colunas
- ais | municipio | arma_utilizada | natureza_fato | data | sexo_vitima | idade_vitima | dummy_contagem

#### Ajustar base de 2013-2017

In [3]:
# Filtrar para eliminar dados incompletos
df_base2013a2017 = df_base2013a2017[df_base2013a2017['IDADE'] != 'NI']

# Criar dataframe auxiliar 1
df_aux1 = pd.DataFrame()
df_aux1['ais'] = df_base2013a2017['AIS']
df_aux1['municipio'] = df_base2013a2017['MUNICÍPIO']
df_aux1['arma_utilizada'] = df_base2013a2017['ARMA UTILIZADA'].replace('ARMA BRANCA', 'Arma branca').replace('ARMA DE FOGO', 'Arma de fogo').replace('OUTROS', 'Outros meios').replace('NÃO INFORMADO', 'Meio não informado')
df_aux1['natureza_fato'] = df_base2013a2017['NATUREZA DO FATO']
df_aux1['data'] = pd.to_datetime(df_base2013a2017['DATA'], format="%Y-%m-%d")
df_aux1['sexo_vitima'] = df_base2013a2017['SEXO']
df_aux1['idade_vitima'] = df_base2013a2017['IDADE'].astype(int)
df_aux1['id_base'] = 1

# df_aux1.dtypes
# df_aux1['arma_utilizada'].unique()
# df_aux1.describe()

#### Ajustar base de 2017-2018
E obter somente os dados de 2018 dessa série

In [4]:
# Ajustar base 2017-2018

# dicionário para mapear os meses
dict_mes = {
    "Janeiro":"01", "Fevereiro":"02", "Março":"03",
    "Abril":"04", "Maio":"05", "Junho":"06",
    "Julho":"07", "Agosto":"08", "Setembro":"09",
    "Outubro":"10", "Novembro":"11", "Dezembro":"12",
}

# Filtrar para eliminar dados incompletos e possíveis duplicados
df_base2017a2018 = df_base2017a2018[df_base2017a2018['ANO'].astype(str) == "2018"] # filtrar somente dados de 2018

# Criar dataframe auxiliar 2
df_aux2 = pd.DataFrame()
df_aux2['ais'] = df_base2017a2018['AIS']
df_aux2['municipio'] = df_base2017a2018['MUNICÍPIO']
df_aux2['arma_utilizada'] = df_base2017a2018['TIPO_ARMA']
df_aux2['natureza_fato'] = df_base2017a2018['NATUREZA']
df_aux2['data'] = pd.to_datetime((df_base2017a2018['ANO'].astype(str) + "-" + df_base2017a2018['MÊS'].replace(dict_mes) + "-01"), format="%Y-%m-%d")
df_aux2['sexo_vitima'] = df_base2017a2018['SEXO'].replace('Masculino', 'MASCULINO').replace('Feminino', 'FEMININO').replace('Não Informado', 'NAO INFORMADO')
df_aux2['idade_vitima'] = df_base2017a2018['IDADE'].astype(str).str.replace('-','0').astype(int)
df_aux2['id_base'] = 2

# df_aux2.dtypes
# df_aux2['arma_utilizada'].unique()
# df_aux2.describe()

#### Ajustar base de 2019-2020

In [5]:
# Ajustar base 2019-2020
df_aux3 = pd.DataFrame()
df_aux3['ais'] = df_base2019a2020['AIS']
df_aux3['municipio'] = df_base2019a2020['MUNICÍPIO']
df_aux3['arma_utilizada'] = df_base2019a2020['ARMA UTILIZADA']
df_aux3['natureza_fato'] = df_base2019a2020['NATUREZA DO FATO']
df_aux3['data'] = df_base2019a2020['DATA']
df_aux3['sexo_vitima'] = df_base2019a2020['SEXO'].replace('Masculino', 'MASCULINO').replace('Feminino', 'FEMININO')
df_aux3['idade_vitima'] = df_base2019a2020['IDADE'].astype(str).str.replace('-','0').astype(int)
df_aux3['id_base'] = 3

# df_aux3.dtypes
# df_aux3['arma_utilizada'].unique()
# df_aux3.describe()

#### Ajustar base de 2021

In [6]:
# Ajustar base parcial de 2021 (até junho)
df_aux4 = pd.DataFrame()
df_aux4['ais'] = df_base2021parcial['AIS']
df_aux4['municipio'] = df_base2021parcial['Município']
df_aux4['arma_utilizada'] = df_base2021parcial['Tipo de Arma']
df_aux4['natureza_fato'] = df_base2021parcial['Natureza']
df_aux4['data'] = df_base2021parcial['Data']
df_aux4['sexo_vitima'] = df_base2021parcial['Gênero'].replace('Masculino', 'MASCULINO').replace('Feminino', 'FEMININO')
df_aux4['idade_vitima'] = df_base2021parcial['Idade da Vítima'].astype(str).str.replace('Não Informada','0').astype(int)
df_aux4['id_base'] = 4

# df_aux4.dtypes
# df_aux4['arma_utilizada'].unique()
# df_aux4.describe()

#### Unificar todas as bases: série de 2013 à 2021

In [7]:
# DataFrame para o estudo
df_estudo = pd.concat([df_aux1, df_aux2, df_aux3, df_aux4])
df_estudo['dummy_contagem'] = 1 # coluna fictícia para contagem

# Escrever arquivo CSV para o estudo
df_estudo.to_csv('../data/processed/dt_estudo.csv', index=False)